In [1]:
# LOAD PACKAGES
# -------------
using JuMP  # Need to say it whenever we use JuMP
using GLPK  # Loading the GLPK module for using its solver

# MODEL CONSTRUCTION
# ------------------
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [2]:
# INPUT DATA
# ----------

n = 3 # no. of (integer) decision variables for x, y, z

# coefficients for constraints
demand_1 = [20; 40; 60] # demand LAX -> DFW
demand_2 = [80; 60; 70] # demand LAX -> JFK
demand_3 = [20; 20; 30] # demand DFW -> JFK 

# coefficients for obf. f.
c = [100; 90; 80]
d = [215; 185; 145]
e = [140; 120; 100]

3-element Vector{Int64}:
 140
 120
 100

In [3]:
# VARIABLES
# ---------

# We work with three vectors of decision variables: x, y, and z
@variable(model, x[1:n]>=0, Int) # LAX -> DFW 
@variable(model, y[1:n]>=0, Int) # LAX -> JFK
@variable(model, z[1:n]>=0, Int) # DFW -> JFK

3-element Vector{VariableRef}:
 z[1]
 z[2]
 z[3]

In [4]:
# CONSTRAINTS
# -----------

# We define a vector of constraints
@constraint(model, sum(x) + sum(y) <= 300)         # capacity of LAX -> DFW
@constraint(model, sum(y) + sum(z) <= 200)         # capacity of DFW -> JFK
@constraint(model, [i = 1:3], x[i] <= demand_1[i]) # demand LAX -> DFW
@constraint(model, [j = 1:3], y[j] <= demand_2[j]) # demand LAX -> JFK
@constraint(model, [l = 1:3], z[l] <= demand_3[l]) # demand DFW -> JFK


3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 z[1] ≤ 20.0
 z[2] ≤ 20.0
 z[3] ≤ 30.0

In [5]:
# OBJECTIVE
# ---------

# Maximize cTx + dTy + eTz
@objective(model, Max, c'x + d'y + e'z)  

# PRINT THE MODEL IN A HUMAN-READABLE FORMAT
# ------------------------------------------
println("The optimization problem to be solved is:")
print(model) # Shows the model constructed in a human-readable form

The optimization problem to be solved is:


In [6]:
# SOLVE IT
# --------

status = optimize!(model) # solves the model  
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [7]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", objective_value(model)) 
println("x = ", value.(x)) 
println("y = ", value.(y)) 
println("z = ", value.(z)) 

Objective value: 47300.0
x = [20.0, 40.0, 60.0]
y = [80.0, 60.0, 40.0]
z = [20.0, 0.0, 0.0]
